In [ ]:
# Forcing autoreload of modules so restart is not required
%load_ext autoreload
%autoreload 2

import os

os.chdir("../../")


print("Current Working Directory:  ", os.getcwd())

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt  
import seaborn as sns
from scipy import stats
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [ ]:
train = pd.read_csv("./data/staging/NB15/train.csv")
test = pd.read_csv("./data/staging/NB15/test.csv")

train.drop('label',axis=1,inplace=True)
test.drop('label',axis=1,inplace=True)

In [4]:
train.head()

,id,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,...,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat
0,1,0.121478,tcp,NaN,FIN,6,4,258,172,74.087490,...,1,1,1,0,0,0,1,1,0,Normal
1,2,0.649902,tcp,NaN,FIN,14,38,734,42014,78.473372,...,1,1,2,0,0,0,1,6,0,Normal
2,3,1.623129,tcp,NaN,FIN,8,16,364,13186,14.170161,...,1,1,3,0,0,0,2,6,0,Normal
3,4,1.681642,tcp,ftp,FIN,12,12,628,770,13.677108,...,1,1,3,1,1,0,2,1,0,Normal
4,5,0.449454,tcp,NaN,FIN,10,6,534,268,33.373826,...,2,1,40,0,0,0,2,39,0,Normal


In [5]:
print(train['attack_cat'].unique())
print(test['attack_cat'].unique())

['Normal' 'Backdoor' 'Analysis' 'Fuzzers' 'Shellcode' 'Reconnaissance'
 'Exploits' 'DoS' 'Worms' 'Generic']
['Normal' 'Reconnaissance' 'Backdoor' 'DoS' 'Exploits' 'Analysis'
 'Fuzzers' 'Worms' 'Shellcode' 'Generic']


In [6]:
print(len(train))
print(len(test))

175341
82332


In [7]:
train['label'] = train['attack_cat'].apply(lambda x: 0 if x == 'Normal' else 1 if x == 'DoS' else 2)
test['label'] = test['attack_cat'].apply(lambda x: 0 if x == 'Normal' else 1 if x == 'DoS' else 2)

In [8]:
train.tail(500)

,id,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,...,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat,label
174841,174842,0.000009,udp,dns,INT,2,0,114,0,111111.107200,...,17,31,0,0,0,17,31,0,Generic,2
174842,174843,0.000004,udp,dns,INT,2,0,114,0,250000.000600,...,15,26,0,0,0,26,26,0,Generic,2
174843,174844,0.000007,udp,dns,INT,2,0,114,0,142857.140900,...,11,26,0,0,0,26,26,0,Generic,2
174844,174845,0.000007,udp,dns,INT,2,0,114,0,142857.140900,...,5,31,0,0,0,7,31,0,Generic,2
174845,174846,0.000005,udp,dns,INT,2,0,114,0,200000.005100,...,17,33,0,0,0,34,33,0,Generic,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175336,175337,0.000009,udp,dns,INT,2,0,114,0,111111.107200,...,13,24,0,0,0,24,24,0,Generic,2
175337,175338,0.505762,tcp,NaN,FIN,10,8,620,354,33.612649,...,1,2,0,0,0,1,1,0,Shellcode,2
175338,175339,0.000009,udp,dns,INT,2,0,114,0,111111.107200,...,3,13,0,0,0,3,12,0,Generic,2
175339,175340,0.000009,udp,dns,INT,2,0,114,0,111111.107200,...,14,30,0,0,0,30,30,0,Generic,2


In [9]:
print(len(train))
print(len(test))

175341
82332


In [10]:
print(train['attack_cat'].unique())
print(test['attack_cat'].unique())

['Normal' 'Backdoor' 'Analysis' 'Fuzzers' 'Shellcode' 'Reconnaissance'
 'Exploits' 'DoS' 'Worms' 'Generic']
['Normal' 'Reconnaissance' 'Backdoor' 'DoS' 'Exploits' 'Analysis'
 'Fuzzers' 'Worms' 'Shellcode' 'Generic']


In [13]:
X_train, Y_train = train.drop(columns=['label']), train['label']
X_test, Y_test = test.drop(columns=['label']), test['label']

In [ ]:
x_train = set(X_train.columns.tolist())
x_test = set(X_test.columns.tolist())

diff_train = x_train - x_test 
diff_test = x_test - x_train
print("Columns in X_train but not in X_test:", diff_train)
print("Columns in X_test but not in X_train:", diff_test)

print(X_train.shape, X_test.shape)

Columns in X_train but not in X_test: set()
Columns in X_test but not in X_train: set()
(175341, 44) (82332, 44)


In [15]:
print(train.dtypes.value_counts())
print(test.dtypes.value_counts())

int64      30
float64    11
object      4
Name: count, dtype: int64
int64      30
float64    11
object      4
Name: count, dtype: int64


In [16]:
print(train.select_dtypes(exclude=np.number).columns)
print(test.select_dtypes(exclude=np.number).columns)


Index(['proto', 'service', 'state', 'attack_cat'], dtype='object')
Index(['proto', 'service', 'state', 'attack_cat'], dtype='object')


In [17]:
categories_col = ['proto', 'service', 'state','attack_cat']
num_col_train = list(set(X_train.columns) - set(categories_col))
num_col_test = list(set(X_test.columns) - set(categories_col))

In [18]:
scaler = StandardScaler()
scaler.fit(X_train[num_col_train])

StandardScaler()

In [ ]:
X_train[num_col_train] = scaler.transform(X_train[num_col_train])
X_test[num_col_test] = scaler.transform(X_test[num_col_test])

In [20]:
X_train.drop('attack_cat',axis=1,inplace=True)
X_test.drop('attack_cat',axis=1,inplace=True)

In [21]:
X_train.head()

,id,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,...,ct_dst_ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports
0,-1.732041,-0.191029,tcp,NaN,FIN,-0.104456,-0.135769,-0.049134,-0.102726,-0.576371,...,-0.645013,-0.544736,-0.554373,-0.705529,-0.121892,-0.118590,-0.189768,-0.715714,-0.753074,-0.126508
1,-1.732021,-0.109485,tcp,NaN,FIN,-0.046014,0.172599,-0.046410,0.188544,-0.576345,...,-0.645013,-0.544736,-0.554373,-0.614256,-0.121892,-0.118590,-0.189768,-0.715714,-0.288257,-0.126508
2,-1.732001,0.040699,tcp,NaN,FIN,-0.089845,-0.026933,-0.048527,-0.012133,-0.576734,...,-0.520827,-0.544736,-0.554373,-0.522983,-0.121892,-0.118590,-0.189768,-0.595543,-0.288257,-0.126508
3,-1.731982,0.049729,tcp,ftp,FIN,-0.060624,-0.063212,-0.047016,-0.098563,-0.576737,...,-0.520827,-0.544736,-0.554373,-0.522983,8.204011,7.814915,-0.189768,-0.595543,-0.753074,-0.126508
4,-1.731962,-0.140417,tcp,NaN,FIN,-0.075235,-0.117630,-0.047554,-0.102057,-0.576617,...,-0.520827,-0.420468,-0.554373,2.854115,-0.121892,-0.118590,-0.189768,-0.595543,2.779535,-0.126508


In [22]:
X_test.head()

,id,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,...,ct_dst_ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports
0,-1.732041,-0.209773,udp,NaN,INT,-0.133677,-0.172047,-0.047772,-0.103923,-0.027189,...,-0.645013,-0.544736,-0.554373,-0.614256,-0.121892,-0.11859,-0.189768,-0.715714,-0.660111,-0.126508
1,-1.732021,-0.209774,udp,NaN,INT,-0.133677,-0.172047,-0.040528,-0.103923,0.178922,...,-0.645013,-0.544736,-0.554373,-0.614256,-0.121892,-0.11859,-0.189768,-0.715714,-0.660111,-0.126508
2,-1.732001,-0.209774,udp,NaN,INT,-0.133677,-0.172047,-0.044499,-0.103923,0.632367,...,-0.645013,-0.544736,-0.554373,-0.522983,-0.121892,-0.11859,-0.189768,-0.715714,-0.567147,-0.126508
3,-1.731982,-0.209774,udp,NaN,INT,-0.133677,-0.172047,-0.045460,-0.103923,0.430836,...,-0.520827,-0.420468,-0.554373,-0.522983,-0.121892,-0.11859,-0.189768,-0.595543,-0.567147,-0.126508
4,-1.731962,-0.209773,udp,NaN,INT,-0.133677,-0.172047,-0.038445,-0.103923,0.027774,...,-0.520827,-0.420468,-0.554373,-0.522983,-0.121892,-0.11859,-0.189768,-0.595543,-0.567147,-0.126508


In [ ]:
service_ohe = OneHotEncoder(handle_unknown="ignore")
proto_ohe = OneHotEncoder(handle_unknown="ignore")
state_ohe = OneHotEncoder(handle_unknown="ignore")

ohe_service_train = service_ohe.fit(X_train[['service']])
ohe_proto_train = proto_ohe.fit(X_train[['proto']])
ohe_state_train = state_ohe.fit(X_train[['state']])

X_train_service = pd.DataFrame(ohe_service_train.transform(X_train[['service']]).todense(), 
							   columns=ohe_service_train.get_feature_names_out(['service']))
X_train_proto = pd.DataFrame(ohe_proto_train.transform(X_train[['proto']]).todense(), 
							 columns=ohe_proto_train.get_feature_names_out(['proto']))
X_train_state = pd.DataFrame(ohe_state_train.transform(X_train[['state']]).todense(), 
							 columns=ohe_state_train.get_feature_names_out(['state']))

X_train = X_train.drop(['service', 'proto', 'state'], axis=1)
X_train = pd.concat([X_train, X_train_service, X_train_proto, X_train_state], axis=1)

X_test_service = pd.DataFrame(ohe_service_train.transform(X_test[['service']]).todense(), 
							  columns=ohe_service_train.get_feature_names_out(['service']))
X_test_proto = pd.DataFrame(ohe_proto_train.transform(X_test[['proto']]).todense(), 
							columns=ohe_proto_train.get_feature_names_out(['proto']))
X_test_state = pd.DataFrame(ohe_state_train.transform(X_test[['state']]).todense(), 
							columns=ohe_state_train.get_feature_names_out(['state']))

X_test = X_test.drop(['service', 'proto', 'state'], axis=1)
X_test = pd.concat([X_test, X_test_service, X_test_proto, X_test_state], axis=1)

In [24]:
X_train.head()

,id,dur,spkts,dpkts,sbytes,dbytes,rate,sttl,dttl,sload,...,proto_zero,state_CON,state_ECO,state_FIN,state_INT,state_PAR,state_REQ,state_RST,state_URN,state_no
0,-1.732041,-0.191029,-0.104456,-0.135769,-0.049134,-0.102726,-0.576371,0.703839,1.578100,-0.389897,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-1.732021,-0.109485,-0.046014,0.172599,-0.046410,0.188544,-0.576345,-1.141901,1.560002,-0.389928,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-1.732001,0.040699,-0.089845,-0.026933,-0.048527,-0.012133,-0.576734,-1.141901,1.560002,-0.389964,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-1.731982,0.049729,-0.060624,-0.063212,-0.047016,-0.098563,-0.576737,-1.141901,1.560002,-0.389958,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-1.731962,-0.140417,-0.075235,-0.117630,-0.047554,-0.102057,-0.576617,0.723268,1.560002,-0.389927,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
X_test.head()

,id,dur,spkts,dpkts,sbytes,dbytes,rate,sttl,dttl,sload,...,proto_zero,state_CON,state_ECO,state_FIN,state_INT,state_PAR,state_REQ,state_RST,state_URN,state_no
0,-1.732041,-0.209773,-0.133677,-0.172047,-0.047772,-0.103923,-0.027189,0.723268,-0.720406,0.567591,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,-1.732021,-0.209774,-0.133677,-0.172047,-0.040528,-0.103923,0.178922,0.723268,-0.720406,4.287318,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,-1.732001,-0.209774,-0.133677,-0.172047,-0.044499,-0.103923,0.632367,0.723268,-0.720406,4.146097,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,-1.731982,-0.209774,-0.133677,-0.172047,-0.045460,-0.103923,0.430836,0.723268,-0.720406,2.795469,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,-1.731962,-0.209773,-0.133677,-0.172047,-0.038445,-0.103923,0.027774,0.723268,-0.720406,4.124860,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
X_train = X_train.dropna()
Y_train = Y_train.loc[X_train.index]  

X_test = X_test.dropna()
Y_test = Y_test.loc[X_test.index]

In [ ]:
x_train = set(X_train.columns.tolist())
x_test = set(X_test.columns.tolist())

diff_train = x_train - x_test
diff_test = x_test - x_train

print("Columns in X_train but not in X_test:", diff_train)
print("Columns in X_test but not in X_train:", diff_test)

print(X_train.shape, X_test.shape)

Columns in X_train but not in X_test: set()
Columns in X_test but not in X_train: set()
(175341, 195) (82332, 195)


In [28]:
Y_train = Y_train.to_frame()
Y_test = Y_test.to_frame()

type(Y_test)

pandas.core.frame.DataFrame

In [29]:
Y_train['label'].unique()

array([0, 2, 1])

In [30]:
Y_test['label'].unique()

array([0, 2, 1])

In [31]:
X_train.to_csv('./data/processed/NB15/multi_class/X_train.csv', index=False)
Y_train.to_csv('./data/processed/NB15/multi_class/Y_train.csv', index=False)

In [32]:
X_test.to_csv('./data/processed/NB15/multi_class/X_test.csv', index=False)
Y_test.to_csv('./data/processed/NB15/multi_class/Y_test.csv', index=False)